In [1]:
import pandas as pd
import random
import numpy as np

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)


C:\Users\Isabel\AppData\Local\Temp\ipykernel_13288\3175742525.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('../Fase1_datos/lista_accion.json', 'r') as fichero:
    accion = json.load(fichero)

In [3]:
len(accion)

3679

In [4]:
busqueda = accion[:500]
len(busqueda)

500

In [5]:
# variables donde iremos almacenando todo
info_por_peli = []
fallos = []

In [6]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))


# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]

    try:
    # metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
            
    except:
        fallos.append(peli)

In [7]:
len(info_por_peli)

495

In [8]:
len(fallos)

0

In [9]:
lista_tuplas = []

In [10]:
for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')
    patron_duracion = re.compile(r'\b(?:\d+h\s*)?\d+min\b')
    tupla_datos = [peli[0],] 

    # listas intermedias para ir checkeando
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []
        
    for indice, linea in enumerate(peli[1]):
               
        
        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            
            if r'\d+h' or r'\d+min' in peli[1][indice-1]:  
                duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])

    # añadimos a la tupla los datos de la lista intermedia
    for elemento in [direccion, guion, nota, duracion, argumento]:
        if elemento != []:
            tupla_datos.append(elemento[0])
        else:
            tupla_datos.append(np.nan)
    
    lista_tuplas.append(tupla_datos)

In [11]:
lista_tuplas

[['tt0113092',
  'David Douglas, Tim Douglas',
  'David Douglas, Christopher Salazar',
  3.4,
  '1h 40min',
  'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new cities in ruins. It falls to a bunch of teens to bring peace to the colony - but will they?'],
 ['tt0116391', 'Mazhar Khan', nan, 6.2, '2h 47min', nan],
 ['tt0116628',
  'Kenneth Guertin',
  'Kenneth Guertin, James Bonner',
  6.3,
  '1h 26min',
  "Alan Freeling, an idealistic inventor, discovers the ultimate energy source. Upon doing so he opens the door to a covert agency that funnels its every resource into destroying Alan's life and hiding this new technology. Their objective is to erase Alan and his family by any means necessary. In a turn of events, Robert Skotak, one of the top officials at the agency, de... Leer todo"],
 ['tt0119273',
  'Michael

In [11]:
with open('AccionWIP.json', 'w') as fichero:
    json.dump(lista_tuplas, fichero)

In [10]:
len(fallos)

0